# Fron Dynamic Programming  To ML

In [13]:
origin_price = [1,5,8,9,10,17,17,20,24,30,33] # 长度1-11的价格

from collections import defaultdict
prices = defaultdict(int)

for i in range(1,len(origin_price)+1):
    prices[i] = origin_price[i-1]
    
# prices = {i+1: p for i,p in enumerate(origin_price)}

In [14]:
prices

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 33})

In [15]:
solution = {}

def r(n): # the revenue of length n
    candidates = []
    for i in range(1,n):
        candidates.append((r(i) + r(n-i),i))
    candidates.append((prices[n],0))
    
    max_price,split_point = max(candidates,key = lambda x:x[0])
    
    global solution
    solution[n] = (split_point,n-split_point)
    
    return max_price

In [19]:
%%time
r(8)

CPU times: user 2.67 ms, sys: 13 µs, total: 2.69 ms
Wall time: 2.7 ms


22

In [20]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6)}

## decorator

In [27]:
from datetime import datetime
import time

In [30]:
def func1(n):
    begin = datetime.now()
    time.sleep(0.1)
    # print('used time = {}'.format(datetime.now()-begin))
    return n

In [31]:
func1(0)

used time = 0:00:00.102765


0

In [32]:
def get_func_time(func):
    
    def _wrap(n):
        begin = datetime.now()
        result = func(n)
        print('used time = {}'.format(datetime.now()-begin))
        
        return result
    return _wrap

In [35]:
func1_with_time = get_func_time(func1)
'''
@another_func
def some_func:
    pass

==> some_func = anthoer_func(some_func)
'''

func1_with_time(9)

used time = 0:00:00.100086
used time = 0:00:00.100235


9

In [36]:
@get_func_time
def get(n):
    print(n)

In [37]:
get(8)

8
used time = 0:00:00.000172


In [38]:
def memo(func):
    
    cache = {}
    def _wrap(n):
        if n in cache: result = cache[n]
        else:
            result = func(n)
            cache[n] = result
        return result
    
    return _wrap

In [41]:
solution = {}
@memo
def r(n): # the revenue of length n
#     candidates = []
#     for i in range(1,n):
#         candidates.append((r(i) + r(n-i),i))
#     candidates.append((prices[n],0))
    
#     max_price,split_point = max(candidates,key = lambda x:x[0])
    max_price,split_point = max([(prices[n],0)] + [(r(i)+r(n-i),i) for i in range(1,n)],key = lambda x:x[0])
    
    global solution
    solution[n] = (split_point,n-split_point)
    
    return max_price

In [42]:
%%time
r(8)

CPU times: user 50 µs, sys: 0 ns, total: 50 µs
Wall time: 52.9 µs


22

## Dynamic Programming

In [44]:
%%time
r(155)

CPU times: user 7.51 ms, sys: 266 µs, total: 7.78 ms
Wall time: 7.85 ms


465

In [45]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (0, 11),
 12: (2, 10),
 13: (2, 11),
 14: (3, 11),
 15: (2, 13),
 16: (6, 10),
 17: (6, 11),
 18: (2, 16),
 19: (2, 17),
 20: (10, 10),
 21: (10, 11),
 22: (11, 11),
 23: (2, 21),
 24: (2, 22),
 25: (3, 22),
 26: (6, 20),
 27: (6, 21),
 28: (6, 22),
 29: (2, 27),
 30: (10, 20),
 31: (10, 21),
 32: (10, 22),
 33: (11, 22),
 34: (2, 32),
 35: (2, 33),
 36: (3, 33),
 37: (6, 31),
 38: (6, 32),
 39: (6, 33),
 40: (10, 30),
 41: (10, 31),
 42: (10, 32),
 43: (10, 33),
 44: (11, 33),
 45: (2, 43),
 46: (2, 44),
 47: (3, 44),
 48: (6, 42),
 49: (6, 43),
 50: (10, 40),
 51: (10, 41),
 52: (10, 42),
 53: (10, 43),
 54: (10, 44),
 55: (11, 44),
 56: (2, 54),
 57: (2, 55),
 58: (3, 55),
 59: (6, 53),
 60: (10, 50),
 61: (10, 51),
 62: (10, 52),
 63: (10, 53),
 64: (10, 54),
 65: (10, 55),
 66: (11, 55),
 67: (2, 65),
 68: (2, 66),
 69: (3, 66),
 70: (10, 60),
 71: (10, 61

In [43]:
def not_cut(n): return n==0
def parse_solution(target_length,revenue_solution):
    left,right = revenue_solution[target_length]
    
    if not_cut(left): return [right]
    
    return parse_solution(left,revenue_solution)+parse_solution(right,revenue_solution)

In [49]:
parse_solution(37,solution)

[6, 10, 10, 11]

## 动态规划
解决具备一定限制条件的最优问题
- 问题可分成若干子问题
- 子问题之间存在overlapping
- 对重合子问题进行cache
- 根据cache数据加速解决问题
- 还要构建parse_solution函数解析最终结果

e.g. 编辑距离 edit distance ; 相似基因判断 ； 路径规划 
